In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import geopandas as gpd
import pandas as pd
import tifffile

import segtraq as st

PATH = Path("/g/huber/projects/CODEX/segtraq/valid_testdata/BC_cellseg_10x/BC_Xenium_cellseg_sample")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Reading files into spatialdata

Before assessing the quality of a segmentation, we first need to get the data into `spatialdata` format. The following sections show how this can be achieved from a variety of different segmentation methods.

We always require a transcript dataframe, and any additional data can come as shapes, labels, or images.

In [73]:
# reading the transcript df
transcript_df = pd.read_csv(PATH / "transcripts.csv")

# optional, if you want to add an image
image = tifffile.imread(PATH / "dapi_um.tif")

transcript_df.head()

,transcript_id,cell_id,overlaps_nucleus,feature_name,x_location,y_location,z_location,qv,fov_name,nucleus_distance,codeword_index,codeword_category,is_gene
0,282643208155248,bfnbkogm-1,0,AAMP,57.153320,163.29980,20.156250,35.75,V12,0.15625,3277,predesigned_gene,True
1,282643207975264,UNASSIGNED,0,ABCA1,69.262695,175.47168,23.562500,40.00,V12,0.93750,9629,predesigned_gene,True
2,282716222414197,nhlaipjn-1,1,ABCA1,73.387695,189.01855,21.593750,40.00,W12,0.00000,9629,predesigned_gene,True
3,282643207864202,bfndjloi-1,0,ABCA1,100.747070,141.87793,20.546875,36.25,V12,0.93750,9629,predesigned_gene,True
4,282716222540753,nhlcfdpj-1,1,ABCA7,85.856445,192.45605,22.890625,40.00,W12,0.00000,8133,predesigned_gene,True


If you want to load the data into a spatialdata object yourself, you can use the `create_spatialdata()` method. Alternatively, you can opt for one of our technology-specific readers, which are detailed below.

In [89]:
# example for how to create a spatialdata object yourself
sdata = st.fs.create_spatialdata(
    points=transcript_df, images=image, coord_columns=["x_location", "y_location", "z_location"]
)
sdata

INFO     no axes information specified in the object, setting `dims` to: ('c', 'y', 'x')                           


SpatialData object
├── Images
│     └── 'image': DataArray[cyx] (1, 551, 680)
└── Points
      └── 'transcripts': DataFrame with shape: (<Delayed>, 13) (3D points)
with coordinate systems:
    ▸ 'global', with elements:
        image (Images), transcripts (Points)

If you already have a spatialdata object, you can quickly assess its consistency with `validate_spatialdata()`. This will check if the cell IDs match between the transcripts and the shapes, if the labels and shapes contain the same number of cells, and a couple of other things. If everything is okay, the method will simply return `True`.

In [90]:
st.fs.validate_spatialdata(sdata)

True

## Technology Focus: ProSeg

In [74]:
shapes = gpd.read_file(PATH / "proseg_output/cell-polygons-layers.geojson")

In [75]:
# reading the nucleus shapes and converting the data frames into geopandas dfs
# TODO
# nucleus_shapes = st.fs.create_geopandas_df(nucleus_shapes)

In [77]:
# TODO: FIX THE MAPPING OF CELL IDS AND LABEL IDS
# sdata = st.fs.create_spatialdata(points=transcript_df,
#                                 images=image,
#                                 shapes=shapes,
#                                 coord_columns=['x_location', 'y_location', 'z_location'],
#                                 cell_key_shapes='cell',
#                                 relabel_shapes=True)

In [ ]:
# TODO: where is the mapping between cell IDs and cell labels?
# shape_df = gpd.read_file('/g/huber/projects/CODEX/segtraq/valid_testdata/BC_cellseg_10x/BC_Xenium_cellseg_sample/
# proseg_output/cell-polygons-layers.geojson')
# sdata = st.fs.create_spatialdata(points=transcript_df, shapes=shape_df,
# optional, if your coordinates are not named x, y, z
#                                 coord_columns=['x_location', 'y_location', 'z_location'],
#                                 cell_key_shapes='cell',  # optional, if your cell is not called cell_id
#                                 relabel_shapes=True,  # optional, if your cell IDs do not start at 1
#                                )
# sdata

## Segger

## BIDcell

In [78]:
bidcell_path = PATH / "bidcell_output"

# reading cell labels
cell_labels_path = list(bidcell_path.glob("model_outputs/202*/test_output/epoch_4_step_100_connected.tif"))
cell_labels = tifffile.imread(cell_labels_path[0])

# reading nucleus labels
nucleus_labels = tifffile.imread(bidcell_path / "nuclei.tif")

# reading the resized image
image = tifffile.imread(bidcell_path / "dapi_resized.tif")

# reading the processed transcripts
transcripts = pd.read_csv(bidcell_path / "transcripts_processed.csv", index_col=0)

In [79]:
sdata = st.fs.create_spatialdata(
    points=transcripts,
    labels={"cell_labels": cell_labels, "nucleus_labels": nucleus_labels},
    images=image,
    # optional, if your coordinates are not named x, y, z
    coord_columns=["x_location", "y_location", "z_location"],
)
sdata

INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                
INFO     no axes information specified in the object, setting `dims` to: ('c', 'y', 'x')                           


SpatialData object
├── Images
│     └── 'image': DataArray[cyx] (1, 551, 680)
├── Labels
│     ├── 'cell_labels': DataArray[yx] (551, 680)
│     └── 'nucleus_labels': DataArray[yx] (551, 680)
└── Points
      └── 'transcripts': DataFrame with shape: (<Delayed>, 13) (3D points)
with coordinate systems:
    ▸ 'global', with elements:
        image (Images), cell_labels (Labels), nucleus_labels (Labels), transcripts (Points)

In [80]:
# we can compute the shapes as follows
sdata = st.fs.compute_shapes(sdata, labels_key="cell_labels")
sdata

SpatialData object
├── Images
│     └── 'image': DataArray[cyx] (1, 551, 680)
├── Labels
│     ├── 'cell_labels': DataArray[yx] (551, 680)
│     └── 'nucleus_labels': DataArray[yx] (551, 680)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 13) (3D points)
└── Shapes
      └── 'cell_boundaries': GeoDataFrame shape: (2276, 2) (2D shapes)
with coordinate systems:
    ▸ 'global', with elements:
        image (Images), cell_labels (Labels), nucleus_labels (Labels), transcripts (Points), cell_boundaries (Shapes)

In [82]:
# TODO: FIX THE MAPPING FROM CELL TO LABEL IDS
# st.fs.validate_spatialdata(sdata, cell_key_points='cell_id')

## Xenium

In [83]:
image = tifffile.imread(PATH / "dapi_um.tif")
cell_labels = tifffile.imread(PATH / "cell_mask_um.tif")
nucleus_labels = tifffile.imread(PATH / "nuc_mask_um.tif")
cell_shapes = pd.read_parquet(PATH / "cell_boundaries.parquet")
nucleus_shapes = pd.read_parquet(PATH / "nucleus_boundaries.parquet")
transcript_df = pd.read_csv(PATH / "transcripts.csv")

In [84]:
# converting the data frames into geopandas dfs
cell_shapes = st.fs.create_geopandas_df(cell_shapes)
nucleus_shapes = st.fs.create_geopandas_df(nucleus_shapes)

In [85]:
sdata = st.fs.create_spatialdata(
    points=transcripts,
    labels={"cell_labels": cell_labels, "nucleus_labels": nucleus_labels},
    images=image,
    shapes={"cell_boundaries": cell_shapes, "nucleus_boundaries": nucleus_shapes},
    # optional, if your coordinates are not named x, y, z
    coord_columns=["x_location", "y_location", "z_location"],
    consolidate_shapes=True,
)
sdata

INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                
INFO     no axes information specified in the object, setting `dims` to: ('c', 'y', 'x')                           


/g/huber/users/meyerben/notebooks/spatial_transcriptomics/SegTraQ/src/segtraq/fs/fix_spatialdata.py:299: UserWarning: Missing 7 cell IDs in shapes: There are 2231 cell IDs in labels, but only 2224 are in shapes. This might lead to inconsistencies in the spatialdata object.
  validate_spatialdata(


SpatialData object
├── Images
│     └── 'image': DataArray[cyx] (1, 551, 680)
├── Labels
│     ├── 'cell_labels': DataArray[yx] (551, 680)
│     └── 'nucleus_labels': DataArray[yx] (551, 680)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 13) (3D points)
└── Shapes
      ├── 'cell_boundaries': GeoDataFrame shape: (2198, 2) (2D shapes)
      └── 'nucleus_boundaries': GeoDataFrame shape: (2195, 2) (2D shapes)
with coordinate systems:
    ▸ 'global', with elements:
        image (Images), cell_labels (Labels), nucleus_labels (Labels), transcripts (Points), cell_boundaries (Shapes), nucleus_boundaries (Shapes)

In [86]:
# ensuring that the new spatialdata is consistent
st.fs.validate_spatialdata(sdata, cell_key_points="cell_id")

/tmp/ipykernel_3517/3303122172.py:2: UserWarning: Missing 33 cell IDs in shapes: There are 2231 cell IDs in labels, but only 2198 are in shapes. This might lead to inconsistencies in the spatialdata object.
  st.fs.validate_spatialdata(sdata, cell_key_points='cell_id')


True